In [46]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }
:dep serde = { version = "1.0.103", features = ["derive"] }
:dep serde_json = "1.0.85"
extern crate plotters;
extern crate serde;
extern crate serde_json;
// Import all the plotters prelude functions
use plotters::prelude::*;
use serde::{Serialize, Deserialize};


#[derive(Serialize, Deserialize)]
pub struct CircSerde {
    pub guard: String,
    pub middle: String,
    pub exit: String,
}

let tor_ps_path = "circs_torps_test.txt";
let tor_ps_circs_str = std::fs::read_to_string(&tor_ps_path).unwrap();
let tor_ps_circs : Vec<CircSerde> = serde_json::from_str(&tor_ps_circs_str).unwrap();


evcxr_figure((320,50), |root| {
    root.fill(&GREEN)?;
    root.draw(&Text::new("Hello World from Plotters!", (15, 15), ("Arial", 20).into_font()))?;
    Ok(())
})

The type of the variable relay_map was redefined, so was lost.
The type of the variable tor_ps_circs was redefined, so was lost.
The type of the variable tor_gen_circs was redefined, so was lost.


Hello World from Plotters!

In [47]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep serde = { version = "1.0.103", features = ["derive"] }
:dep serde_json = "1.0.85"
extern crate plotters;
extern crate serde;
extern crate serde_json;
// Import all the plotters prelude functions
use plotters::prelude::*;
use serde::{Serialize, Deserialize};
use std::collections::HashMap;


#[derive(Serialize, Deserialize)]
pub struct CircSerde {
    pub guard: String,
    pub middle: String,
    pub exit: String,
}
#[derive(Default)]
pub struct SimulationSource{
    gen: usize,
    torps: usize
}
fn circs_to_map(gen_circs: &Vec<CircSerde>, torps_circs: &Vec<CircSerde>) -> HashMap<String, SimulationSource> {
    let mut tor_relays_map : HashMap<String, SimulationSource> = HashMap::new();
    for circ in gen_circs {
        tor_relays_map.entry(circ.guard.clone()).or_default().gen += 1;
        tor_relays_map.entry(circ.middle.clone()).or_default().gen += 1;
        tor_relays_map.entry(circ.exit.clone()).or_default().gen += 1;
    }
    for circ in torps_circs {
        tor_relays_map.entry(circ.guard.clone()).or_default().torps += 1;
        tor_relays_map.entry(circ.middle.clone()).or_default().torps += 1;
        tor_relays_map.entry(circ.exit.clone()).or_default().torps += 1;
    }
    tor_relays_map    
}

fn map_to_combined_histo(map: &HashMap<String,SimulationSource>) -> (Vec<(usize,usize,usize)>, usize){
    let mut histo : Vec<(usize,usize,usize)> = vec![];
    let mut index = 0;
    let mut max = 0;
    for (relay, source) in map {
        if source.gen > max {
            max = source.gen;
        }
        if source.torps > max {
            max = source.torps;
        }
        histo.push((index, source.gen, source.torps));
        index += 1;
    }
    (histo, max)
}
let tor_ps_path = "circs_torps";
let tor_gen_path = "circs_generated";
let tor_ps_circs_str = std::fs::read_to_string(&tor_ps_path).unwrap();
let tor_gen_circs_str = std::fs::read_to_string(&tor_gen_path).unwrap();
let tor_ps_circs : Vec<CircSerde> = serde_json::from_str(&tor_ps_circs_str).unwrap();
let tor_gen_circs : Vec<CircSerde> = serde_json::from_str(&tor_gen_circs_str).unwrap();
let relay_map = circs_to_map(&tor_gen_circs, &tor_ps_circs);
let (relay_histo, max_count) = map_to_combined_histo(&relay_map);

evcxr_figure((640, 480), |root| {
    let areas = root.split_evenly((2,1));
    let mut charts = vec![];
    
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&areas[0])
            .caption(format!("Histogram for TorPS"), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0usize..relay_histo.len(), 0usize..max_count)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .y_labels(5)
            .draw()?;
        charts.push(chart);
    let mut chart = ChartBuilder::on(&areas[1])
            .caption(format!("Histogram for Tor Circuit Generator"), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0usize..relay_histo.len(), 0usize..max_count)?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .y_labels(5)
            .draw()?;
        charts.push(chart);
    
    charts[0].draw_series(
        relay_histo.iter().map(|(idx, gen, torps)| Circle::new((*idx,*gen), 1, &BLUE)),
    )
    .unwrap();
    
       charts[1].draw_series(
        relay_histo.iter().map(|(idx, gen, torps)| Circle::new((*idx,*torps), 1, &RED)),
    )
    .unwrap();

    Ok(())
}).style("width:100%")

Histogram for TorPS
 
 
 
0
 
 
 
5000
 
 
 
10000
 
 
 
 
0
 
 
 
500
 
 
 
1000
 
 
 
1500
 
 
 
2000
 
 
 
2500
 
 
 
3000
 
 
 
3500
 
 
 
4000
 
 
 
4500
 
 
 
Histogram for Tor Circuit Generator
 
 
 
0
 
 
 
5000
 
 
 
10000
 
 
 
 
0
 
 
 
500
 
 
 
1000
 
 
 
1500
 
 
 
2000
 
 
 
2500
 
 
 
3000
 
 
 
3500
 
 
 
4000
 
 
 
4500
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 


In [57]:
fn map_to_compared_histo(map: &HashMap<String,SimulationSource>) -> (Vec<(usize,usize,usize)>, Vec<String>, usize){
    let mut histo : Vec<(usize,usize,usize)> = vec![];
    let mut fingerprints : Vec<String> = vec![];
    let mut index = 0;
    let mut max = 0;
    for (relay, source) in map {
        let diff = source.gen.abs_diff(source.torps);
        if diff > 500 {
            if (diff > max) {
                max = diff;
            }
            histo.push((index, source.gen, source.torps));
            fingerprints.push(relay.to_string());
            index += 1;
        }
    }
    (histo, fingerprints, max)
}

let (compared_histo, fingerprints, max) = map_to_compared_histo(&relay_map);

evcxr_figure((480, 3000), |root| {
    let mut charts = vec![];
    
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
            .caption(format!("Compared Histogram for TorPS"), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_cartesian_2d(0usize..max, 0usize..compared_histo.len())?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
             .y_label_formatter(&|x| format!("{:.3}", x))
            .y_labels(10)
            .draw()?;
        charts.push(chart);
    charts[0].draw_series(
        compared_histo.iter().map(|(idx, gen, torps)| Cross::new((*gen, *idx,), 3, &BLUE)),
    )
    .unwrap();
    charts[0].draw_series(
        compared_histo.iter().map(|(idx, gen, torps)| Circle::new((*torps, *idx), 3, &RED)),
    )
    .unwrap();
    Ok(())
}).style("width:100%")

Compared Histogram for TorPS
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
 
0
 
 
 
1000
 
 
 
2000
 
 
 
3000
 
 
 
4000
 
 
 
5000

In [86]:
fn map_to_compared_histo(map: &HashMap<String,SimulationSource>) -> (Vec<(usize,usize,usize)>, Vec<String>, usize){
    let mut histo : Vec<(usize,usize,usize)> = vec![];
    let mut fingerprints : Vec<String> = vec![];
    let mut index = 0;
    let mut max = 0;
    for (relay, source) in map {
        let diff = source.gen.abs_diff(source.torps);
        if diff > 500 {
             if source.gen > max {
                max = source.gen;
            }
            if source.torps > max {
                max = source.torps;
            }
            histo.push((index, source.gen, source.torps));
            fingerprints.push(relay.to_string());
            index += 1;
        }
    }
    fingerprints.push(String::from(""));
    (histo, fingerprints, max)
}

let (compared_histo, fingerprints, max) = map_to_compared_histo(&relay_map);

evcxr_figure((900, 3000), |root| {
    let mut charts = vec![];
    
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
            .caption(format!("Compared Histogram for TorPS"), ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(250)
            .build_cartesian_2d(0usize..(max+50), 0usize..compared_histo.len())?;
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
             .y_label_formatter(&|x| format!("{}", fingerprints[*x]))
            .y_labels(fingerprints.len())
            .draw()?;
        charts.push(chart);
    charts[0].draw_series(
        compared_histo.iter().map(|(idx, gen, torps)| Cross::new((*gen, *idx,), 3, &BLUE)),
    )
    .unwrap();
    charts[0].draw_series(
        compared_histo.iter().map(|(idx, gen, torps)| Circle::new((*torps, *idx), 3, &RED)),
    )
    .unwrap();
    Ok(())
}).style("width:100%")

Compared Histogram for TorPS
 
 
 
8b8e30b0a49ebeef963d2a5aac28f3bd3068adec
 
 
 
a0296ddc9ec50aa42ed9d477d51dd4607d7876d3
 
 
 
acaae1708144f6485aaffd59c3c6eaa1ea4aaab2
 
 
 
42896cf699a0e9d28b690901925b410eb22d94a6
 
 
 
44be4949cc96603ccfd29b6f79153df641e9ee89
 
 
 
25009d9fa55d89619747995a9087f4d66ce86270
 
 
 
aaab2519c27ef687e3216d588cc2c6acc3b051a3
 
 
 
c8de26d2d119be7f509e672ee1a3d5fb25dcba97
 
 
 
f40016c5a2d7460da5ccbf8a2346135d6bbc3dd0
 
 
 
9a338f384c5fd3a3604e3250ccf5f5f7629d7717
 
 
 
37d354ee6b22d6fb0ee9921874c0578059d69013
 
 
 
824fddadce9b893382e28ba54c52406dbe3740c3
 
 
 
9acd21680ccbe65a5d5a638d85a8388a85d6ee05
 
 
 
f08a525aca965cf6a55f0587bf4b7373d98b954e
 
 
 
1694a8cd4564d987c59d8a17d74f153ed967ac0c
 
 
 
e4bfad49d21d3838485c62d84293a97daaaa35b7
 
 
 
23722976b04384e41ae71b45e040051ae6e398f2
 
 
 
fa54bdbf6054b3f52281e993b1c01ebbf9c59873
 
 
 
4deaa21675f356da442e288c905c90aad6d24c47
 
 
 
c5b8fbc0237f4a04f2055128f257dff6e9c145b7
 
 
 
191c826ea91b9fdf45ff50e01db2b9cfae7114ee
 
 
 
15291291d81e404fb6bec16d366608590d2b0247
 
 
 
61fbd0b66f559af50f5593e791b0318f7fcd1d74
 
 
 
9835d50d56c7871a3c1ba1c8a86865d4810cdc70
 
 
 
5197fc89f7a1623ca90d6e0254abccbc6d85a86e
 
 
 
58a2a0e849e9e3692f7b136223507e93cb099150
 
 
 
d07b7b9185119985debbb2f4e19f6a19ea746ced
 
 
 
2835071f4b86d28df923d2aa2ba902f28fa4f285
 
 
 
ac7c0f9d57dadad5d8f4568ee1543ef3e22a47ce
 
 
 
6f475da2b81db6a790289136b28e23ba94e84b23
 
 
 
8f14815fb60bbb23e4bc46e162e37c1428ea1ccd
 
 
 
5278b03c88a3a992387b4b615161b8006c4a65ad
 
 
 
9d66ecaed38e54d784cd5717703df83022fb64f4
 
 
 
ecfdbcdff24cd0a2d4cece93cc51eb74a5a16bbd
 
 
 
633d3673e20d6fbc2c4e6e2931d60b0de6a5cd75
 
 
 
070b03eb62bd79a9794b903f3a38aebba25ddef9
 
 
 
b670b0be157cc2a19b76f04341b5f8e9950d4550
 
 
 
c7106f0c6a7cf0f592cbaff7c79bd81153b1b5f5
 
 
 
1702f6888757d679afbec9842f4b98e9b02b6d40
 
 
 
ae6ce2b402c2930ebaf59a616e80ad43f7ab123b
 
 
 
204775e9a2fa4928e1589b9e6307cb3d104e8499
 
 
 
2fcbf27febdc4fefc3714c44885acd8352791e76
 
 
 
fa1845c313b88ef49e8be1f23b02cd0ac702b636
 
 
 
b3fdd767eef15dd75dc8d25e5f6ae92d9a2353b9
 
 
 
4d70f05dd308a2ea5b671e440879bfadac92860f
 
 
 
ab1febf69822f4aa142c7031924dd08496cbfeea
 
 
 
72cf2fe156f8637a21acb7d163a9a395d0c02043
 
 
 
fbd20387f64d67ed53ac5bd2e0302d44b2da360c
 
 
 
6055fe90c18dd4b2593a9d0e02ddc4d68e9ba62e
 
 
 
19c9906fcb64b12a55d8b381f5731a05120e5fd4
 
 
 
8caa470b905758742203e3eb45941719fca9feec
 
 
 
2a82dff7beb7b4269d4d6eae39dec36f98214268
 
 
 
7751e37017f7666883daf2503b3014182828536a
 
 
 
a4e47f08b8d56428df76b17edd6738bcbc3f5efb
 
 
 
148487c6ebe04eb22496080c20796c51b060f611
 
 
 
c1939d36649de98a202429631d8efc70128d5f5f
 
 
 
3a4e6203b16d291a7e2a2e262b062dab70e538e1
 
 
 
72faae8f5fe6992131b9465860672fdc9bf31d09
 
 
 
c3f7f3e1e32a64b22b2f0734e7c7a312f993d102
 
 
 
e7fb7f21d68b3e598c098e3540b6965dc8d934fc
 
 
 
8ee44717fa55705c12086f3ecd1f8d9c8676fd05
 
 
 
d8056cad4a3d6ee7d82748027474bf5c53e569d9
 
 
 
0d5aa7c0c53b65ebe9ce3a4093cbaf2319c45271
 
 
 
72ab7c961cd2aeeb949474caf88df5221546d2bf
 
 
 
047e96e51be832fc4e95fe399a437e18b957b6b3
 
 
 
833024b3c4893af10663483f28c51e6ca6bd6508
 
 
 
5e4d1e6d31413dccc148a8050224578cbbf12883
 
 
 
4c25542b7649fc0472edbf77521ea578f9af5155
 
 
 
0e69aa0e00ab9e5bbf052c864682229b0b1f16ad
 
 
 
73a08ceb49a213fc73ffd97389638487d4f1bb71
 
 
 
3a608f27b4d65675a319862dd7f8e8441e0b4a64
 
 
 
e1d7adbf52626819d499eb4a9e3ef1808e9242e7
 
 
 
c0c4f339046eb824999f711d178472fdf53be7f5
 
 
 
22bcd0dfd148209c9860c7f89907ab4dee974a08
 
 
 
509eab4c5d10c9a9a24b4ea0ce402c047a2d64e6
 
 
 
5a79bd5cc6c128d7d8dfb4969b0246794f117fc6
 
 
 
6dced82b701d323120811df58d350dd26bf9cb25
 
 
 
a8eb70983cc2fc3e2c91419e84ebcd4722654742
 
 
 
0ec0515743b4c2883f908c528774a6e12e613b85
 
 
 
ca991ad48b046808780fa07867c4882365391632
 
 
 
bb5fae50bce5b13c810cd17a931a0498e4681d41
 
 
 
f5f8497b39c1022bca786012390e1fda55ab73ec
 
 
 
74b0055d8540458402fe0f548fc6a844b5431a97
 
 
 
52bfada8beaa01ba46c8f767f83c18e2fe50c1b9
 
 
 
182bff0100b977269